In [1]:
# importing libraries
import pandas as pd # Reading csv file 
from shapely.geometry import Point # Shapely for converting latitude/longtitude to geometry
import geopandas as gpd # To create GeodataFrame

In [81]:
data_url = "total_trips_2017.csv"
trip = pd.read_csv(data_url)

In [82]:
# creating a geometry column 
geometry = [Point(xy) for xy in zip(trip['start_lon'], trip['start_lat'])]

# Coordinate reference system : WGS84
crs = {'init': 'epsg:4269'}

# Creating a Geographic data frame 
gdf = gpd.GeoDataFrame(trip, crs=crs, geometry=geometry)

In [83]:
data_url = "/Users/kjeong/1_Work/1_SmartMobility/Data/tl_2016_us_county/tl_2016_us_county.shp"
zone_fips=gpd.read_file(data_url)
data_url = "/Users/kjeong/1_Work/1_SmartMobility/Data/tl_2016_us_county/faf_129_4269.shp"
zone_FAF=gpd.read_file(data_url)

In [84]:
zone_fips=zone_fips[['GEOID','geometry']]
zone_FAF=zone_FAF[['FAFZ','geometry']]

In [85]:
join = gpd.sjoin(gdf, zone_fips, how="left", op="within")
join2 = gpd.sjoin(gdf, zone_FAF, how="left", op="within")

In [86]:
trip['org_fips']=join['GEOID']
trip['org_FAF']=join2['FAFZ']

In [87]:
# creating a geometry column 
geometry = [Point(xy) for xy in zip(trip['end_lon'], trip['end_lat'])]

# Coordinate reference system : WGS84
crs = {'init': 'epsg:4269'}

# Creating a Geographic data frame 
gdf = gpd.GeoDataFrame(trip, crs=crs, geometry=geometry)

In [88]:
join = gpd.sjoin(gdf, zone_fips, how="left", op="within")
join2 = gpd.sjoin(gdf, zone_FAF, how="left", op="within")
trip['dest_fips']=join['GEOID']
trip['dest_FAF']=join2['FAFZ']

In [89]:
trip=trip.drop(columns=['geometry'])

In [90]:
def org(row):
    if row['org_FAF'] in (171,181):
        val = row['org_fips']
    else: val = row['org_FAF']
    return val
def dest(row):
    if row['dest_FAF'] in (171,181):
        val = row['dest_fips']
    else: val = row['dest_FAF']
    return val  

In [91]:
trip['org_zone'] = trip.apply(org, axis=1)
trip['dest_zone'] = trip.apply(dest, axis=1)
trip.to_csv ('/Users/kjeong/1_Work/1_SmartMobility/Data/INRIX/trips_zones.csv', index = None, header=True)

In [92]:
sum_trip=trip.groupby(['org_zone','dest_zone','vehicle_weight_class']).agg({'trip_id':'count'})

In [93]:
sum_trip.columns = ["".join(x) for x in sum_trip.columns.ravel()]
sum_trip.reset_index(level=(0,1,2), inplace=True)

In [94]:
sum_trip.to_csv ('/Users/kjeong/1_Work/1_SmartMobility/Data/INRIX/sum_trip.csv', index = None, header=True)

In [95]:
sum_trip.head()

,org_zone,dest_zone,vehicle_weight_class,trip_id
0,11,179,2,1
1,11,269,2,1
2,11,559,2,1
3,11,17031,2,3
4,11,17031,3,1


In [115]:
data_url = "/Users/kjeong/1_Work/1_SmartMobility/Delivarables/19Q2Report/Chicago_flows/Chicago_inbound.csv"
Chi_in = pd.read_csv(data_url)
data_url = "/Users/kjeong/1_Work/1_SmartMobility/Delivarables/19Q2Report/Chicago_flows/Chicago_outbound.csv"
Chi_out = pd.read_csv(data_url)

In [114]:
Chi_out.head()

,sctg2,dms_orig,dms_dest,fips,SU1,SU2,SU3,SU4,SU5,SU6,...,DBL9,TPT1,TPT2,TPT3,TPT4,TPT5,TPT6,TPT7,TPT8,TPT9
0,1,171,61,17197,0,0,0.0,0.0,0.0,0.0,...,0,0,0,0.0,0.0,0,0,0,0,0
1,1,171,61,17037,0,0,0.0,0.0,0.0,0.0,...,0,0,0,0.0,0.0,0,0,0,0,0
2,1,171,61,17093,0,0,0.0,0.0,0.0,0.0,...,0,0,0,0.0,0.0,0,0,0,0,0
3,1,171,61,17099,0,0,0.0,0.0,0.0,0.0,...,0,0,0,0.0,0.0,0,0,0,0,0
4,1,171,61,17099,0,0,0.0,0.0,0.0,0.0,...,0,0,0,0.0,0.0,0,0,0,0,0


In [123]:
Chi_in_v2 = pd.DataFrame(columns=['org_zone', 'dest_zone', 'num_veh'])
Chi_out_v2 = pd.DataFrame(columns=['org_zone', 'dest_zone', 'num_veh'])
Chi_in_v2['org_zone']= Chi_in['dms_orig']
Chi_in_v2['dest_zone']= Chi_in['fips']
Chi_in_v2['num_veh'] =Chi_in.iloc[:,4:48].sum(axis=1)
Chi_out_v2['org_zone']= Chi_out['fips']
Chi_out_v2['dest_zone']= Chi_out['dms_dest']
Chi_out_v2['num_veh'] =Chi_out.iloc[:,4:48].sum(axis=1)

In [124]:
Chi=pd.concat([Chi_in_v2,Chi_out_v2], axis=0, join='outer', ignore_index=False, keys=None,
          levels=None, names=None, verify_integrity=False, copy=True)

In [126]:
sum_chi=Chi.groupby(['org_zone','dest_zone']).agg({'num_veh':'sum'})

In [127]:
sum_chi.columns = ["".join(x) for x in sum_chi.columns.ravel()]
sum_chi.reset_index(level=(0,1), inplace=True)

In [129]:
sum_trip=sum_trip.groupby(['org_zone','dest_zone']).agg({'trip_id':'sum'})
sum_trip.columns = ["".join(x) for x in sum_trip.columns.ravel()]
sum_trip.reset_index(level=(0,1), inplace=True)

In [130]:
sum_chi

,org_zone,dest_zone,num_veh
0,11,17007,127.953894
1,11,17031,10858.935665
2,11,17037,68.336521
3,11,17043,1170.334558
4,11,17063,24.130215
5,11,17089,690.395026
6,11,17091,79.301773
7,11,17093,21.906448
8,11,17097,760.971067
9,11,17099,35.491371
